In [1]:
import copy
import numpy as np 
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
import networkx as nx
import string
import pandas as pd
import json
import random as rd
from scipy.stats import beta
from itertools import product
from tqdm import tqdm
from joblib import Parallel, delayed

import sys
sys.path.insert(0, '..')
from simulator.fishbeliefs import *

# Infer structure

In [2]:
# Generate stimuli for pilot experiment
real_sequences = []
for k,e in structures.items():
    simulator = FishBeliefNetwork(n_agents=3, edge_p=1/3., planet=1., edges=e, structure=k)
    for _ in range(10):
        simulator.step()
    data = [[step[1]["belief"],step[1]["strength_discrete"],step[2]["belief"],step[2]["strength_discrete"]] for step in simulator.signal_history]   
    real_sequences.append([k,data])



In [3]:
def simulate_data(edges, structure):
    simulator = FishBeliefNetworkNaive(n_agents=3, edge_p=1/3., edges=edges, structure=structure)
    for _ in range(10):
        simulator.step()
    data = [[step[1]["belief"],step[1]["strength_discrete"],step[2]["belief"],step[2]["strength_discrete"]] for step in simulator.signal_history]
    return [structure, data]


tmp = simulate_data(structures['indep'],"indep")


In [4]:
data_size = 1000000
simulated_sequences = []
for k,e in structures.items():
    batch = Parallel(n_jobs=2)(
            delayed(simulate_data)(e,k)
            for i in tqdm(range(data_size)))
    simulated_sequences += batch

        



100%|██████████| 1000000/1000000 [09:18<00:00, 1790.98it/s]


In [5]:
struct_inf = {}
struct_inf_norm = {}
for true_structure, y_obs in real_sequences:
    steps = {}
    unnorm_post = {'indep':0, 'b->c':0, 'c->b':0, 'b<->c':0}
    for sim_structure, y_sim in simulated_sequences:
        # Only looking at exact match of the observed data
        if y_obs == y_sim: 
            unnorm_post[sim_structure] += 1
    struct_inf[true_structure] = unnorm_post

In [6]:
struct_inf

{'indep': {'indep': 30, 'b->c': 0, 'c->b': 0, 'b<->c': 0},
 'b->c': {'indep': 0, 'b->c': 8, 'c->b': 0, 'b<->c': 0},
 'c->b': {'indep': 0, 'b->c': 0, 'c->b': 23, 'b<->c': 0},
 'b<->c': {'indep': 0, 'b->c': 0, 'c->b': 0, 'b<->c': 32}}

In [ ]:
# Unnormalised
for k,v in struct_inf.items():
    print("True structure: {}".format(k))
    for step,samples in v.items():
        print("   Step {}: {}".format(step, samples.items()))
    print()

In [ ]:
# Normalised
for k,v in struct_inf.items():
    print("True structure: {}".format(k))
    for step,samples in v.items():
        print("   Step {}: {}".format(step, {k:np.round(v/sum(samples.values()),2) for k,v in samples.items()}))
    print()

In [ ]:
matplotlib